### Importing Libraries
Importing necessary libraries for building, training, and evaluating a deep learning model using Keras and TensorFlow. It also brings in tools for image preprocessing and visualization

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import matplotlib.pyplot as plt


Defines the path to the training and testing datasets and the size to which all images will be resized

In [ ]:

train_dir = r"C:\Users\kmurm\Downloads\aiml-gc-2025\AI-ML GC 2025 Dataset\train"
test_dir = r"C:\Users\kmurm\Downloads\aiml-gc-2025\AI-ML GC 2025 Dataset\test"
img_size = (224, 224)


### Training Data Augmentation

In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Using 20% data for validation
)



### Training Generator
Sets up the generator for the training subset of the data

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 7637 images belonging to 200 classes.


### Validation Generator

In [21]:
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1777 images belonging to 200 classes.


### Test Generator

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
 shuffle=False
)

### Model Architecture (Transfer Learning with MobileNetV2)

In [ ]:

class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(200, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

## Initial Training (Frozen Base)

In [ ]:

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(
    train_generator,
    epochs=13,
    validation_data=val_generator,
    callbacks=[early_stop]
)

Epoch 1/13
239/239 ━━━━━━━━━━━━━━━━━━━━ 128s 518ms/step - accuracy: 0.7207 - loss: 0.9337 - val_accuracy: 0.1508 - val_loss: 10.0443
Epoch 2/13
239/239 ━━━━━━━━━━━━━━━━━━━━ 115s 480ms/step - accuracy: 0.7697 - loss: 0.7424 - val_accuracy: 0.1902 - val_loss: 10.5898
Epoch 3/13
239/239 ━━━━━━━━━━━━━━━━━━━━ 114s 477ms/step - accuracy: 0.7965 - loss: 0.6457 - val_accuracy: 0.1643 - val_loss: 11.3284
Epoch 4/13
239/239 ━━━━━━━━━━━━━━━━━━━━ 114s 478ms/step - accuracy: 0.8225 - loss: 0.5569 - val_accuracy: 0.1649 - val_loss: 11.5200


## Fine-Tuning (Unfreezing Last 20 Layers)

In [26]:

for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stop]
)

Epoch 1/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 123s 501ms/step - accuracy: 0.8560 - loss: 0.4703 - val_accuracy: 0.3056 - val_loss: 6.1185
Epoch 2/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 118s 493ms/step - accuracy: 0.8870 - loss: 0.3578 - val_accuracy: 0.4322 - val_loss: 3.7019
Epoch 3/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 118s 492ms/step - accuracy: 0.9073 - loss: 0.3048 - val_accuracy: 0.5250 - val_loss: 2.6607
Epoch 4/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 117s 491ms/step - accuracy: 0.9252 - loss: 0.2572 - val_accuracy: 0.5661 - val_loss: 2.2323
Epoch 5/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 121s 506ms/step - accuracy: 0.9283 - loss: 0.2437 - val_accuracy: 0.5954 - val_loss: 1.9237
Epoch 6/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 121s 504ms/step - accuracy: 0.9374 - loss: 0.2305 - val_accuracy: 0.6134 - val_loss: 1.7750
Epoch 7/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 120s 502ms/step - accuracy: 0.9391 - loss: 0.2047 - val_accuracy: 0.6089 - val_loss: 1.7950
Epoch 8/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 116s 487ms/step - accuracy: 0.9377 -

## Saving the model

In [27]:
model.save("bird_classifier_model.keras")

## Load Model and Preprocessing Function

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model("bird_classifier_model.keras")

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = img_array / 255.0
    return img_array


### Predict and Save to CSV

In [29]:
import os
import csv
import numpy as np

submission_data = []


for img_file in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_file)
    img_tensor = preprocess_image(img_path)
    predictions = model.predict(img_tensor)

    predicted_index = int(np.argmax(predictions))  
    

    submission_data.append([img_file, predicted_index])

with open("submission.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "label"])  
    writer.writerows(submission_data)

print("Predictions saved to submission.csv")



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 708ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━